In [ ]:
HTML("""
<iframe src='https://asaragga.github.io/GRF.jl/' width=1000 height=350></iframe>
    """)

## 1. Instalar e Aceder a Aplicações Essenciais

### 1.1 Instalar Aplicações

Utilização remota, e.g. Binder: 

  * Mantêr todos os # 

Se fôr a primeira utilização local deste notebook :
  * Caso Julia Pro tenha sido instalada, apenas retirar os # na primeira e na segunda linha
  * Caso contrário, retirar # em todas as linhas (ou apenas naquelas relativas a aplicações ainda não instaladas)

In [ ]:
# import Pkg 
# Pkg.add("TimeSeries")
# Pkg.add("Distributions")      
# Pkg.add("StatsBase")          
# Pkg.add("HypothesisTests")    
# Pkg.add("StatsPlots")         
# Pkg.add("Interact")           

### 1.2 Tornar as Aplicações Essenciais Disponíveis

In [ ]:
using LinearAlgebra, Distributions, StatsBase, HypothesisTests, StatsPlots, Interact, TimeSeries

## 2. Obter Dados

In [ ]:
VPortfolio = 74.6                                               # Valor do portfolio em USD 1x10^6
cambio = 0.884680                                               # Câmbio USD/EUR no final do dia 2019-04-15
dados = readtimearray("Dow-Jones.csv")                          # Tabela com retornos(∆P/P) diários

1. Nº Observações = 1567 no período de 2013-01-02 a 2019-04-15

2. Nº Colunas = 33

Colunas```:BT3m, :SP500, :MMM, :AXP, :AAPL, :BA, :CAT, :CVX, :CSCO, :KO, :DWDP, :XOM, :GS, :IBM, :INTC, :JNJ, :JPM,:MCD, :MRK, :MSFT, :NKE, :PFE, :PG, :HD, :TRV, :UTX, :UNH, :VZ, :V, :WBA, :WMT, :DIS, :USDEUR```
  * Coluna  1 = ```:BT3m``` taxa de juro dos bilhetes tesouro dos EUA a 3 meses no mercado secundário, em termos diários
  * Coluna  2 = ```:SP500``` retorno simples diário do índice S&P500
  * Colunas 3-32 = ```:MMM,...,:DIS``` retorno simples diário das 30 ações constituintes do índice Dow-Jones em 31 Março 2019
  * Coluna  33 = ```:USDEUR``` taxa de variação cambial diária USD/EUR, com USD/EUR representando o valor de 1 USD em EUR

Por conveniência, iremos guardar os nomes das séries/colunas na vetor ```simbolos(33x1)``` 

In [ ]:
simbolos = colnames(dados)    

Igualmente, iremos guardar as datas das observações no vetor imaginativamente chamado ```datas(1567x1)``` 

In [ ]:
datas = timestamp(dados)

## 3. Visualizar Dados

### 3.1 Séries Temporais de Retornos

In [ ]:
plot(dados[:SP500])         # Retornos diários do S&P500, caracterizados por fases de elevada e baixa volatilidade 

Os retornos do S&P500, apresentam fases de elevada e baixa volatilidade. Se os mercadores forem eficiente não devamos esperar que os valores esperados dos retornos estejam autocorrelacionados. Tal, não impede porém que as **volatilidades** dos retornos possam estar autocorrelacionadas, o que parece acontecer neste caso. Esta situação é aliás uma regularidade típica no mercado acionista.

In [ ]:
plot(plot(dados[:MMM]),plot(dados[:GS]),plot(dados[:MCD]),plot(dados[:DIS]))  # 3M, Goldman-Sachs,McDonalds,Disney

In [ ]:
plot(dados[:USDEUR])        # Variação percentual diária da taxa de câmbio USD/EUR 

### 3.2 Histogramas

In [ ]:
R = values(dados)           # Transforma a Tabela dados(1567x33) na Matriz R(1567x33) equivalente 

  * ```R[:,j]``` seleciona todos os elementos da coluna ```j``` da matriz ```R```
  * ```R[i,:]``` seleciona todos os elementos da linha ```i``` da matriz ```R```
  * ```String.(simbolos[i])``` seleciona o elemento ```i``` do vetor simbolos e transforma-o em tipo Texto (String) permitindo a sua inclusão em gráficos

In [ ]:
plot(
histogram(R[:, 2], nbins=100, title = String.(simbolos[ 2]), xlabel = "Retorno Diário S&P500", legend = false),
histogram(R[:,13], nbins=100, title = String.(simbolos[13]), xlabel = "Retorno Diário Goldman-Sachs", legend = false),
histogram(R[:,18], nbins=100, title = String.(simbolos[18]), xlabel = "Retorno Diário McDonalds", legend = false),
histogram(R[:,32], nbins=100, title = String.(simbolos[32]), xlabel = "Retorno Diário Walt Disney", legend = false))

### 3.3 Dependência Linear: Correlações

In [ ]:
plot(scatter(R[:,2],R[:, 3], title = "3M vs S&P500"           , markersize = 2, legend = false),
     scatter(R[:,2],R[:,13], title = "Goldman-Sachs vs S&P500", markersize = 2, legend = false),
     scatter(R[:,2],R[:,18], title = "McDonalds vs S&P500"    , markersize = 2, legend = false),
     scatter(R[:,2],R[:,32], title = "Walt Disney vs S&P500"  , markersize = 2, legend = false))

In [ ]:
println("Correlação($(simbolos[ 3]),$(simbolos[2])) = " , cor(R[:, 3],R[:,2]))  # Correlação(MMM,S&P500)...
println("Correlação($(simbolos[13]), $(simbolos[2])) = ", cor(R[:,13],R[:,2]))  
println("Correlação($(simbolos[18]),$(simbolos[2])) = " , cor(R[:,18],R[:,2]))  
println("Correlação($(simbolos[32]),$(simbolos[2])) = " , cor(R[:,32],R[:,2]))  

In [ ]:
scatter(R[:,2],R[:,33], legend=false, markersize=2, title="USD/EUR vs. S&P 500", xlabel="S&P500",ylabel="USD/EUR") 

Retorno diário do S&P 500 e da taxa câmbio USD/EUR não parecem estar correlacionados

In [ ]:
println("Correlação($(simbolos[33]),$(simbolos[2])) = ", cor(R[:,33],R[:,2]))   # Correlação(USD/EUR,S&P500)

## 4. Composição do Portfólio

Definir matriz ```W(33x1)``` com os pesos no portfólio, sujeita às seguintes restrições:
  * ```0 <= w_i <= 1```      ausência de short-selling e financiamento do portfólio por empréstimos 
  * ```0 <= w_BT3m  <= 0.05```    peso máximo aplicado em BTs = 5%
  * ```0 <= w_SP500 <= 0.25```    peso máximo aplicado em instrumento replicador do S&P500 = 25%
  * Nº de açôes > 10, sendo o peso mínimo investido em cada >= 2.5%
  * ```sum(w_i) = 1```       todo o portfólio terá de ser aplicada em USD

In [ ]:
W = zeros(33)   # Preencher W com zeros inicialmente (32 oportunidades de investimento + câmbio USD/EUR)
W[1] = 0.0      #  0.0% investido em BTs (indice 1 de W)
W[2] = 0.1      # 10.0% num investimento replicando S&P500 (indice 2 de W)

for i = 3:32    # 90.0% remanescentes investidos em partes iguais pelas 30 ações do Down-Jones (indices 3-32 de W)
    W[i] = 0.03
end             # Notemos que temos já W[33]=0. O câmbio USD/EUR irá ser utilizado depois para a análise em EUR 

In [ ]:
show(W)         # Verificar se é o que queremos
sum(W)          # A soma dos pesos w_i deverá ser um valor muito próximo ou igual a 1

## 5. Simulação Histórica

### 5.1 Simular Historicamente os Retornos do Portfolio Atual

Simulação histórica dos retornos diários do portfolio: ```RP(1567x1) = R(1567x33) * W(33x1)```

In [ ]:
RP = R * W      

In [ ]:
histogram(RP, nbins=100, title = "Portfólio", xlabel = "Retorno Diário", legend = false)

### 5.2 Abordagem Não-Paramétrica ($\mathrm{np}$)

$$\mathrm{VaR} = Z_\alpha V_p$$ onde $$Z_\alpha$$ é o $\alpha$-quantil da **Distribuição Empírica** dos retornos obtidos por simulação histórica. A função ```quantile(X,alfa)``` calcula o ```alfa```-quantil de ```X```.

In [ ]:
VaR05np = - quantile(RP,0.05) * VPortfolio    # O sinal (-) reflete o facto do VaR ser uma medida de perda    
VaR01np = - quantile(RP,0.01) * VPortfolio 

println("DEaR(5%) simulação histórica não paramétrica = ", VaR05np, " USD 1x10^6")
println("DEaR(1%) simulação histórica não paramétrica = ", VaR01np, " USD 1x10^6")

Estes DEaRs refletem o desempenho do mercado acionista no período em que a simulação histórica ocorreu. O valor esperado ($\mu$) dos retornos futuros poderá contudo ser bastante diverso. Se considerarmos que o **valor esperado** do retorno diário é aproximadamente 0, então podemos antes calcular o VaR centrado onde se assume que $\mu=0$.  

In [ ]:
VaR05np = - (quantile(RP,0.05) - mean(RP)) * VPortfolio    
VaR01np = - (quantile(RP,0.01) - mean(RP)) * VPortfolio 

println("DEaR(5%) simulação histórica não paramétrica (com mu=0) = ", VaR05np, " USD 1x10^6")
println("DEaR(1%) simulação histórica não paramétrica (com mu=0) = ", VaR01np, " USD 1x10^6")

### 5.3 Cálculo do VaR do Portfolio em EUR, por Simulação Histórica Não-Paramétrica ($\textrm{np}$)

Denotamos por $$\mathrm{rp}_t^\mathrm{EUR}=\ln(V_t^\mathrm{EUR}/V_{t-1}^\mathrm{EUR}),\qquad \mathrm{rp}^\mathrm{USD}_t=\ln(V^\mathrm{USD}_t/V^\mathrm{USD}_{t-1}), \qquad \mathrm{r}^\mathrm{USDEUR}_t=\ln(C^\mathrm{USDEUR}_t/C^\mathrm{USDEUR}_{t-1})$$ 

os retornos logarítmicos do portfolio em EUR e USD e da taxa de câmbio USD/EUR, respectivamente. Dado que,

$$\mathrm{rp}^\mathrm{EUR} = \mathrm{rp}^\mathrm{USD} + \mathrm{r}^\mathrm{USDEUR}\qquad (\text{5.3.a})$$   

e podemos passar de retornos simples ($\mathrm{R}$) para retornos logarítmicos ($\mathrm{r}$) e vice-versa, através das expressões

$$ \mathrm{r} = \ln(\mathrm{R}+1) \qquad\;\, (\text{5.3.b})$$ 

$$\mathrm{R} = \exp(\mathrm{r})-1 \qquad (\text{5.3.c})$$

irá ser fácil efetuar uma simulação histórica do valor do portfolio em EUR e calcular o VaR. Apenas temos de transformar retornos simples em logarítmicos, usar a equação (5.3.a) e claro, não esquecer de cambiar o valor do portfolio atual de USD para EUR!

O modificador ```.``` aplica um operador (e.g. ```+```, ```*```, ```/```, ```exp```, ```log```, ```sin```,...) a todos os elementos de um vetor/matriz. Exemplo: sendo ```x = [1.0,2.0,3.0]```, tem-se:  
  * ```x.^2 = [1.0^2, 2.0^2, 3.0^2] = [1.0,4.0,9.0]```
  * ```exp.(x) = [exp(1.0),exp(2.0),exp(3.0)] = [2.7182818284590,7.389056098930,20.0855369231876]``` 
  * ```log.(x .+ 1.0) = [log(1.0)+1.0,log(2.0)+1.0,log(3.0)+1.0] = [0.69314718055994,1.09861228866810,1.38629436111989]```

De referir ainda que, como é habitual em linguagens de programação, o $\mathrm{ln(x)}$ é calculado através da função ```log(x)```

In [ ]:
rp_USD = log.(RP .+ 1)              # Equação 5.3.b, retornos simples portfolio USD -> retornos logaritmicos
r_USDEUR = log.(R[:,33] .+ 1)       # Equação 5.3.b, retornos simples USDEUR (coluna 33) -> retornos logaritmicos
rp_EUR = rp_USD + r_USDEUR          # Equação 5.3.a, retornos logaritmicos portfolio EUR
RP_EUR = exp.(rp_EUR) .- 1          # Equação 5.3.c, retornos logaritmicos portfolio EUR -> retornos simples

VPortfolio_EUR = VPortfolio * cambio  

VaR05np_EUR = - (quantile(RP_EUR,0.05) - mean(RP_EUR)) * VPortfolio_EUR
VaR01np_EUR = - (quantile(RP_EUR,0.01) - mean(RP_EUR)) * VPortfolio_EUR

println("DEaR(5%) em EUR por simulação histórica não paramétrica (com mu=0) = ", VaR05np_EUR, " EUR 1x10^6")
println("DEaR(1%) em EUR por simulação histórica não paramétrica (com mu=0) = ", VaR01np_EUR, " EUR 1x10^6")

### 5.4 Abordagem Semi-Paramétrica ($\mathrm{sp}$)

$$\text{VaR}_\alpha = - Z_\alpha V_p$$ com $$Z_\alpha = z_\alpha \sigma_p $$ onde o $\alpha$-quantil da distribuição empírica $Z_\alpha$ é agora determinado pelo produto de dois componemtes: ($\textrm{i}$) quantil $z_\alpha$ da distribuição padronizada escolhida, por exemplo $z_{5\%}$ será igual a -1.645 se a distribuição padronizada for a $\mathcal{N}(0,1)$ e ($\textrm{ii}$) $\sigma_p$ determinado por simulação histórica dos retornos do portfólio.

Iremos pois tentar encontrar em (5.3.1) uma distribuição analítica (e.g ```Normal```, ```T-Student```) que se ajuste bem à distribuição empírica dos retornos diários do portfólio já calculados em (5.1) por simulação histórica.

#### 5.4.1 Escolha da Distribuição Analítica

Padronizamos a distribuição empírica de ```RP``` para a comparar com várias distribuições analíticas candidatas, como a Normal(0,1) ou T-Student com diversos graus de liberdade.

In [ ]:
mu = mean(RP)                 # média dos retornos do portfólio
sigma = std(RP)               # desvio-padrão dos retornos do portfólio

RP01 = (RP .- mu)./sigma      # retornos do portfólio já padronizados, com média = 0 e desvio-padrão = 1         

In [ ]:
histogram(RP01, nbins=100, normed=true, label="Empírica Padronizada")
plot!(Normal(0,1), label = "Normal(0,1)" )
plot!(TDist(6), label = "t-Student(6)")

In [ ]:
plot(TDist(6), label = "t-Student(6)")
plot!(Normal(0,1), label = "Normal(0,1)")

Conseguimos ter uma melhor visualização do que se passa nas caudas das distribuições através de um gráfico quantil-quantil (QQ). A função ```qqplot(Distribuição,X)```permite essa análise. Escolhemos analisar as distribuições Normal(0,1) e T-Student(g) com g = 5, 6, 7, 8, 9.

In [ ]:
plot(                             
qqplot(Normal,   RP01, markersize=1, title = "Normal"),     # Normal(0,1)  vs. distribuição empírica padronizada
qqplot(TDist(5), RP01, markersize=1, title = "TDist(5)"),   # T-Student(5) vs. distribuição empírica padronizada
qqplot(TDist(6), RP01, markersize=1, title = "TDist(6)"),           
qqplot(TDist(7), RP01, markersize=1, title = "TDist(7)"),
qqplot(TDist(8), RP01, markersize=1, title = "TDist(8)"),
qqplot(TDist(9), RP01, markersize=1, title = "TDist(9)")
)

In [ ]:
@manipulate for obs = 1:10
        qqplot(TDist(obs), RP01, markersize=2, title = "Distribuição Emprírica vs. T-Student com $(obs) graus liberdade",
     legend = :bottomright,
     xlabel = "Distribuição T-Student($(obs))", 
     ylabel = "Distribuição Empírica",
     label = "Observações")
end    
# Mover o cursor (obs) no topo do gráfico seguinte para variar os graus de liberdade da distribuição T-Student!

Visualmente, a distribuição que melhor se parece ajustar à distribuição empírica é ```TDist(7)```, a T-Student(7). Teremos porém de utilizar testes de hipóteses para melhor ponderar a nossa escolha.

O teste exato de Kolmogorov–Smirnov ```ExactOneSampleKSTest(X,d)```  testa a hipótese nula de que os dados num determinado vetor ```X``` provêm de uma distribuição ```d```, contra a hipótese alternativa de que a amostra não provém de ```d```.

In [ ]:
println("Teste KS Normal(0,1): ",ExactOneSampleKSTest(RP01, Normal(0,1)))
println("Teste KS TDist(6): ", ExactOneSampleKSTest(RP01, TDist(6)))
println("Teste KS TDist(7): ", ExactOneSampleKSTest(RP01, TDist(7)))
println("Teste KS TDist(8): ", ExactOneSampleKSTest(RP01, TDist(8)))

Todas as hipóteses nulas foram rejeitadas! De facto, embora a T-Student(7) revele um excelente ajustamento nos quantis mais baixos, o ajustamneto nos quantis mais altos é algo fraco. Não obstante, estando nós interessados no ajustamento aos quantis mais baixos (1%-5%) iremos selecionar a T-Student(7). Sabemos porém que se quiséssemos analisar os cenários de melhor desempenho do portfólio teríamos de escolher outra parametrização/distribuição.

#### 5.4.2 Cálculo do VaR a 5% e 1%

In [ ]:
Q05sp =     # Quantis para a distribuição empírica original (não padronizada) 
Q01sp = sigma * quantile(TDist(7),0.01)     # com mu = 0. 

In [ ]:
VaR05sp = - quantile(TDist(7),0.05) * sigma * VPortfolio                             # com mu = 0
VaR01sp = - quantile(TDist(7),0.01) * sigma * VPortfolio                             # com mu = 0

println("DEaR(5%) simulação histórica semi-paramétrica = ", VaR05sp, " USD 1x10^6")
println("DEaR(1%) simulação histórica semi-paramétrica = ", VaR01sp, " USD 1x10^6")

### 5.5 Abordagem Semi-Paramétrica: EWMA

Como vimos no caso do S&P 500, os retornos gerados por ações são caracterizados tipicamente por fases **persistentes** de elevada e reduzida volatilidade. Se, por exemplo, estivermos numa fase de volatilidade elevada será mais provável que a volatilidade dos retornos no futuro próximo se mantenha elevada do que seja semelhante à volatilidade média histórica. 

A análise anterior (5.2) não modelizou a evolução da volatilidade, não incorporando pois esta sua característica de persistência. Uma forma de o fazer, será projetar a volatilidade futura dando um maior peso à informação proveniente das observações mais recentes. Tal é o caso das médias móveis ponderadas exponencialmente (EWMA - exponentially weighted moving average).

##  6. Abordagem Paramétrica: Full Model ($\mathrm{fm}$)

$$\text{VaR}_\alpha = -z_\alpha \sigma_p V_p \qquad (\text{6.1})$$ onde $$\sigma_P = \sqrt{\mathrm{W}^\top \Sigma\, \mathrm{W}}\qquad\; (\text{6.2})$$ e $$W_{(33\times 1)} \text{ e } \Sigma_{(33\times 33)}$$ denotam o vetor de pesos e a matriz de variâncias/covariâncias, respetivamente.

In [ ]:
sigma_fm = (W' * cov(R) * W)^(1/2)                                              # Equação 6.2
VaR05fm = - quantile(TDist(7),0.05) * sigma_fm * VPortfolio                     # Equação 6.1, alpha=0.05
VaR01fm = - quantile(TDist(7),0.01) * sigma_fm * VPortfolio                     # Equação 6.1, alpha=0.01

println("DEaR(5%) abordagem paramétrica full model = ", VaR05fm, " USD 1x10^6")
println("DEaR(1%) abordagem paramétrica full model = ", VaR01fm, " USD 1x10^6")

## 7. Análise de Stress 

Iremos analisar os cenários de stress correspondentes às (1%) piores perdas do portfolio. Depois de criarmos uma nova tabela ```TabelaRP``` apenas com os retornos do portfolio e respetivas datas, iremos identificar as datas em que as piores perdas ocorreram.

In [ ]:
TabelaRP = TimeArray(datas,RP,[:RP]) # Tabela temporal com os retornos simulados do portfólio (coluna :RP) e datas

O passo seguinte será pois determinar para o portfólio quais os dias em que se verificaram retornos simulados $\leq$ 1%-quantil

In [ ]:
stress = TabelaRP[ findall(TabelaRP[:RP] .<= quantile(RP,0.01)) ]  

Certamente o ano de 2018 teria sido bastante stressante para o portfólio. O pior dia foi 5 Fevereiro de 2018, com uma queda de -4.44%. Vamos tentar ver o que poderá ter acontecido de especial nesse dia, quer nos mercados NYSE e NASDAQ, quer nos próprios EUA.

Em 5 Fevereiro 2018, o indíce Dow caiu -4.6%: [Dow suffers worst one-day point fall in history](https://www.youtube.com/watch?v=GJMmK0sAYBk). Mas porque é que os mercados abruptamente cairam tanto nesse dia? Frequentemente não é possivel associar uma causa económica ou política direta aos eventos extremos de 'crash' nas bolsas de ações nos dias precisos em que ocorrem. 

Tal não será porém o caso do dia 10 Outubro 2018, em que se verificou uma queda de -2.99% no valor do portfolio: [World stock markets dive as Trump attacks 'crazy' US rate hikes](https://www.theguardian.com/business/2018/oct/11/asian-stock-markets-dive-as-trump-attacks-crazy-us-rate-hikes).

## 8. Análise de Desempenho do Portfólio

Vamos analisar o desempenho do portfolio face ao desempenho do benchmark escolhido. Medidas de desempenho têm de levar em conta não só o retorno obtido, mas também o risco associado ao investimento no portfólio. Uma medida muito utilizada é o rácio Sharpe, criado por William Sharpe um dos autores do CAPM. O rácio Sharpe ex-ante é definido da seguinte forma:


$$S = \frac{E[R-R_b]}{\sqrt{\mathrm{var}[R-R_b]}}$$

onde $R$ é o retorno do portfólio e $R_b$ é o retorno do seu benchmark. Estamos contudo mais interessados na sua concretização empírica, o rácio Sharpe ex-post, que utiliza os retornos realizados e não o valor esperado de retornos futuros.
Denotando os retornos em excesso do portfólio face ao benchmark (os quais podem ser negativos) por $X_i$,

$$X_i = R_i-R_{bi}$$
$$\mu_x=\tfrac{1}{N}\sum_{i=1}^N X_i$$
e
$$S = \frac{\mu_x}{\sqrt{\tfrac{1}{N}\sum_{i=1}^N (X_i-\mu_x)^2}}$$
onde normalmente se utiliza a variância da amostra.

### 8.1 Benchmark: S&P500

O benchmark natural do portfólio será o índice S&P500:

In [ ]:
X = RP .- R[:,2]                    # Vetor com os retornos em excesso do portfólio face ao S&P 500
Sharpe = mean(X)/std(X)             # Notemos que mean(X) e std(X) são números e não vetores, daí não usamos ./

### 8.2 Benchmark: BT3m

Utilizando agora aplicações em Bilhetes do Tesouro a 3 meses como benchmark do portfólio:

In [ ]:
X = RP .- R[:,1]                    #  Vetor com os retornos em excesso do portfólio face a BT3m
Sharpe = mean(X)/std(X)

### 8.3 Comparar Rácio Sharpe do Portfólio e do S&P500 face a uma aplicação em BT3m

In [ ]:
X_P = RP .- R[:,1]
SharpePortfolio = mean(X_P)/std(X_P)

X_SP500 = R[:,2]-R[:,1]
SharpeSP500 = mean(X_SP500)/std(X_SP500)

println("Rácio Sharpe: Portfolio face BT = ", SharpePortfolio, ", Rácio Sharpe: S&P500 face BT = ", SharpeSP500)